<a href="https://colab.research.google.com/github/snowysunny/Colaboratory/blob/master/tensorflow1_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow 安装

In [0]:
# 不需要进行安装
#! pip install tensorflow-gpu

In [0]:
import tensorflow as tf
tf.enable_eager_execution()

In [0]:
A = tf.constant([[1, 2], [3, 4]])
B = tf.constant([[5, 6],[7, 8]])
C = tf.matmul(A, B)

print(C)

tf.Tensor(
[[19 22]
 [43 50]], shape=(2, 2), dtype=int32)


# Tensorflow 基础

In [0]:
# Tensorflow 1 + 1
import tensorflow as tf
tf.enable_eager_execution

<function tensorflow.python.framework.ops.enable_eager_execution>

In [0]:
# 数值相加
a = tf.constant(1)
b = tf.constant(1)
c = tf.add(a, b)
print(c)

tf.Tensor(2, shape=(), dtype=int32)


In [0]:
# 矩阵相乘
A = tf.constant([[1, 2], [3, 4]])
B = tf.constant([[5, 6],[7, 8]])
C = tf.matmul(A, B)

print(C)

tf.Tensor(
[[19 22]
 [43 50]], shape=(2, 2), dtype=int32)


In [0]:
x = tf.get_variable(name='x', shape=[1], initializer=tf.constant_initializer(3.))

# 在tf.GradientTape()的上下文内，所有计算步骤都会被记录以用于求导
with tf.GradientTape() as tape:
    y = tf.square(x)  # y = x^2
  
# 计算y关于x的导数
y_grad = tape.gradient(y, x)
print([y.numpy(), y_grad.numpy()])

[array([9.], dtype=float32), array([6.], dtype=float32)]


这里x是一个初始化为3的变量(Variable)，使用tf.get_variable()声明。与普通张量一样，变量同样具有形状(shape)和类型(dtype)属性，不过使用变量需要有一个初始化过程，可以通过在tf.get_variable()中指定initializer参数来指定所使用的的初始化器。

变量和普通张量的一个重要区别是其默认能够被Tensorflow的自动求导机制所求导，因此常常被用于定义机器学习模型的参数。tf.GradientTape()是一个自动求导的记录器，在其中变量和计算步骤都会被自动记录。

**使用tf.GradientTape()计算函数$L(w, b) = ||Xw+b-y||^2$在$w=(1,2)^T, b=1$时对$w, b$的偏导数。 其中$ X=\begin{bmatrix} 1 & 2 \\ 3 & 4 \\ \end{bmatrix} , y = \begin{bmatrix} 1 \\ 2 \\ \end{bmatrix} $**

In [0]:
X = tf.constant([[1., 2.], [3., 4.]])
y = tf.constant([[1.], [2.]])
w = tf.get_variable('w', shape=[2, 1], initializer=tf.constant_initializer([[1.], [2.]]))
b = tf.get_variable('b', shape=[1], initializer=tf.constant_initializer([1.]))
with tf.GradientTape() as tape:
    L = 0.5 * tf.reduce_sum(tf.square(tf.matmul(X, w) + b - y))     # tf.reduce_sum()操作代表对输入张量的所有元素求和，输出一个形状为空的纯量张量  ， tf.square()操作代表对输入张亮的每一个元素求平方，不改变张量形状

w_grad , b_grad = tape.gradient(L, [w, b])
print([L.numpy(), w_grad.numpy(), b_grad.numpy()])

[62.5, array([[35.],
       [50.]], dtype=float32), array([15.], dtype=float32)]


# 基础示例： 线性回归

In [0]:
import numpy as np

X_raw = np.array([2013, 2014, 2015, 2016, 2017])
y_raw = np.array([12000, 14000, 15000, 16500, 17500])

X = (X_raw - X_raw.min()) / (X_raw.max() - X_raw.min())
y = (y_raw - y_raw.min()) / (y_raw.max() - y_raw.min())

In [0]:
# 输出参数
X = tf.constant(X)
y = tf.constant(y)
print(X)
print(y)

tf.Tensor([0.   0.25 0.5  0.75 1.  ], shape=(5,), dtype=float64)
tf.Tensor([0.         0.36363636 0.54545455 0.81818182 1.        ], shape=(5,), dtype=float64)


In [0]:
# 权值
w = tf.get_variable('w', shape=[], initializer=tf.zeros_initializer(), dtype=tf.float64)
b = tf.get_variable('b', shape=[], initializer=tf.zeros_initializer(), dtype=tf.float64)
variables = [w, b]
print(variables)

[<tf.Variable 'w:0' shape=() dtype=float64, numpy=0.0>, <tf.Variable 'b:0' shape=() dtype=float64, numpy=0.0>]


In [0]:
num_epoch = 10000
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-3)

In [0]:
for e in range(num_epoch):
    # 使用tf.GradientTape()记录损失函数的梯度信息
    with tf.GradientTape() as tape:
        y_pred = w * X + b
        loss = 0.5 * tf.reduce_sum(tf.square(y_pred - y))
    # Tensorflow 自动计算损失函数关于自变量（模型参数）的梯度
    grads = tape.gradient(loss, variables)
  
    # Tensorflow自动根据梯度更新参数
    optimizer.apply_gradients(grads_and_vars=zip(grads, variables))


In [0]:
print(w.numpy())
print(b.numpy())

0.9763702100237027
0.05756498006354141


>* 使用tape.gradient(ys, xs)自动计算梯度
>* 使用optimizer.apply_gradients(grads_and_vars)自动更新模型参数
>* 使用tf.train.GradientDescentOptimizer(learning_rate=1e-3)声明一个梯度下降优化器(Optimizer)，学习率为1e-3


# 4. TensorFlow 模型



## 4.1 模型与层

In [0]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()

In [0]:
X = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
y = tf.constant([[10.0], [20.0]])

In [0]:
class Linear(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense = tf.keras.layers.Dense(units=1, kernel_initializer=tf.zeros_initializer(), bias_initializer=tf.zeros_initializer())
    
    def call(self, input):
        output = self.dense(input)
        return output

In [0]:
model = Linear()
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
for i in range(100):
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.reduce_sum(tf.square(y_pred-y))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
print(model.variables)

[<tf.Variable 'linear/dense_10/kernel:0' shape=(3, 1) dtype=float32, numpy=
array([[0.16730969],
       [1.1439102 ],
       [2.120511  ]], dtype=float32)>, <tf.Variable 'linear/dense_10/bias:0' shape=(1,) dtype=float32, numpy=array([0.97660077], dtype=float32)>]


## 4.2 基础示例： 多层感知机(MLP)

In [0]:
# 实现一个简单的DataLoader类来读取MNIST数据集数据
class DataLoader():
    def __init__(self):
        mnist = tf.contrib.learn.datasets.load_dataset("mnist")
    
        self.train_data = mnist.train.images      # np.array [55000, 748]
    
        self.train_labels = np.asarray(mnist.train.labels, dtype=np.int32)    # np.array    [55000]  int32

        self.eval_data = mnist.test.images        # np.array [10000, 748]
    
        self.eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)     # np.array      [10000]    int32

    # 批量处理数据
    def get_batch(self, batch_size):
        index = np.random.randint(0, np.shape(self.train_data)[0], batch_size)
        return self.train_data[index, :], self.train_labels[index]
  

In [0]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        # units: 正整数，输出空间维度。
        self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)
    
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return x
    
    def predict(self, inputs):
        logits = self(inputs)
        return tf.argmax(logits, axis=-1)
  

In [0]:
num_batches=10000
batch_size=50    # 批大小
learning_rate = 0.001    # 学习率

In [0]:
# 实例化模型，数据读取类和优化器
data_loader = DataLoader()
model = MLP()
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


然后迭代进行以下步骤：
>* 从DataLoader中随机去除一批训练数据
>* 将这批数据送入模型，计算出模型的预测值
>* 将模型预测值与真实值进行对比，计算损失函数(loss)
>* 计算损失函数关于模型变量的导数
>* 使用优化器更新模型参数以最小化损失函数


In [0]:
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
  
    with tf.GradientTape() as tape:
        y_logit_pred = model(tf.convert_to_tensor(X))
        loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_logit_pred)
        if batch_index %100 == 0:
            print("batch %d : loss %f" %(batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0 : loss 0.342618
batch 100 : loss 0.182857
batch 200 : loss 0.227622
batch 300 : loss 0.257889
batch 400 : loss 0.226254
batch 500 : loss 0.383519
batch 600 : loss 0.218000
batch 700 : loss 0.068599
batch 800 : loss 0.275217
batch 900 : loss 0.148904
batch 1000 : loss 0.142543
batch 1100 : loss 0.320799
batch 1200 : loss 0.141551
batch 1300 : loss 0.118681
batch 1400 : loss 0.071995
batch 1500 : loss 0.220347
batch 1600 : loss 0.042813
batch 1700 : loss 0.046956
batch 1800 : loss 0.133350
batch 1900 : loss 0.077915
batch 2000 : loss 0.040952
batch 2100 : loss 0.124818
batch 2200 : loss 0.152551
batch 2300 : loss 0.266619
batch 2400 : loss 0.031239
batch 2500 : loss 0.098745
batch 2600 : loss 0.051543
batch 2700 : loss 0.334216
batch 2800 : loss 0.066380
batch 2900 : loss 0.048744
batch 3000 : loss 0.044871
batch 3100 : loss 0.045026
batch 3200 : loss 0.055900
batch 3300 : loss 0.077920
batch 3400 : loss 0.118902
batch 3500 : loss 0.085524
batch 3600 : loss 0.015221
batch 3700 : 

In [0]:
num_eval_samples = np.shape(data_loader.eval_labels)[0]

y_pred = model.predict(data_loader.eval_data).numpy()

print("test accuracy: %f" % (sum(y_pred == data_loader.eval_labels) / num_eval_samples))

test accuracy: 0.973200


## 4.3 卷积神经网络(CNN)

In [0]:
class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        # filter: 整数，输出空间的维度 （即卷积中滤波器的输出数量）, kernel_size: 卷积核的大小 padding: padding策略，same表示填充输入以使输出具有与原始输入相同的长度 
        self.conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2,2], strides=2)
        
        self.conv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2,2], strides=2)
        
        self.flatten = tf.keras.layers.Reshape(target_shape=(7*7*64, ))
        self.dense1 = tf.keras.layers.Dense(units=1024, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)
        
    def call(self, inputs):
        inputs = tf.reshape(inputs, [-1, 28, 28, 1])
        x = self.conv1(inputs)     # [batch_size, 28, 28, 32]
        x = self.pool1(x)          # [batch_size, 14, 14, 32]
        x = self.conv2(x)          # [batch_size, 14, 14, 64]
        x = self.pool2(x)          # [batch_size, 7, 7, 64]
        x = self.flatten(x)        # [batch_size, 7 * 7 * 64]
        x = self.dense1(x)         # [batch_size, 1024]
        x = self.dense2(x)         # [batch_size, 10]
        return x
    
    def predict(self, inputs):
        logits = self(inputs)
        return tf.argmax(logits, axis=-1)
    

In [0]:
num_batches=1000
batch_size=50    # 批大小
learning_rate = 0.001    # 学习率

data_loader = DataLoader()
model = CNN()
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [0]:
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    
    with tf.GradientTape() as tape:
        y_logit_pred = model(tf.convert_to_tensor(X))
        loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_logit_pred)
        if batch_index % 100 == 0:
            print("batch %d : loss %f" %(batch_index, loss.numpy()))        
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0 : loss 0.010695
batch 100 : loss 0.068098
batch 200 : loss 0.245348
batch 300 : loss 0.107799
batch 400 : loss 0.015472
batch 500 : loss 0.010678
batch 600 : loss 0.036522
batch 700 : loss 0.079104
batch 800 : loss 0.052215
batch 900 : loss 0.004032


In [0]:
num = np.shape(data_loader.eval_labels)[0]

y_pred = model.predict(data_loader.eval_data).numpy()

print("test accuracy: %f" % (sum(y_pred == data_loader.eval_labels) / num_eval_samples))

test accuracy: 0.987300


In [0]:
num_eval_samples = np.shape(data_loader.eval_labels)[0]

y_pred = model.predict(data_loader.eval_data).numpy()

print("test accuracy: %f" % (sum(y_pred == data_loader.eval_labels) / num_eval_samples))

test accuracy: 0.956500


## 4.4 循环神经网络(RNN)

In [0]:
class DataLoader():
    def __init__(self):
        path = tf.keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
        with open(path, encoding='utf-8') as f:
            self.raw_text = f.read().lower()
            
        self.chars = sorted(list(set(self.raw_text)))
        self.char_indices = dict((c, i) for i , c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i , c in enumerate(self.chars))
        self.text = [self.char_indices[c] for c in self.raw_text]
        
    def get_batch(self, seq_length, batch_size):
        seq = []
        next_char = []
        for i in range(batch_size):
            index = np.random.randint(0, len(self.text) - seq_length)
            seq.append(self.text[index:index+seq_length])
            next_char.append(self.text[index+seq_length])
        return np.array(seq), np.array(next_char)

In [0]:
class RNN(tf.keras.Model):
    def __init__(self, num_chars):
        super().__init__()
        self.num_chars = num_chars
#         self.cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=256)
        self.cell = tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell', num_units=256)
        
        self.dense = tf.keras.layers.Dense(units=self.num_chars)
        
    def call(self, inputs):
        batch_size, seq_length = tf.shape(inputs)
        inputs = tf.one_hot(inputs, depth=self.num_chars)     # [batch_size, seq_length, num_chars]
        state = self.cell.zero_state(batch_size=batch_size, dtype=tf.float32)
        
        for t in range(seq_length.numpy()):
            output, state = self.cell(inputs[:, t, :], state)
        output = self.dense(output)
        return output
    
    def predict(self, inputs, temperature=1.):
        batch_size, _ = tf.shape(inputs)
        logits = self(inputs)
        prob = tf.nn.softmax(logits / temperature).numpy()
        return np.array([np.random.choice(self.num_chars, p=prob[i,:]) for i in range(batch_size.numpy())])
        

训练过程
>* 从DataLoader中随机取一批训练数据
>* 将这批数据送入模型，计算出模型的预测值
>* 将模型预测值与真实值进行比较，计算损失函数(loss)
>* 计算损失函数关于模型变量的导数
>* 使用优化器更新模型参数以最小化损失函数

In [0]:
num_batches = 100
batch_size = 50
seq_length = 1
learning_rate = 0.001
data_loader = DataLoader()

In [0]:
model = RNN(len(data_loader.chars))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)

In [0]:
X, y = data_loader.get_batch(seq_length, batch_size)
print(X, ', ', X.shape)
print(y)

[[ 0]
 [35]
 [35]
 [40]
 [40]
 [ 0]
 [31]
 [46]
 [46]
 [ 0]
 [47]
 [39]
 [ 1]
 [46]
 [ 1]
 [41]
 [40]
 [27]
 [46]
 [40]
 [35]
 [31]
 [ 1]
 [ 0]
 [33]
 [46]
 [51]
 [45]
 [38]
 [41]
 [ 1]
 [40]
 [35]
 [31]
 [41]
 [34]
 [ 1]
 [35]
 [31]
 [34]
 [ 0]
 [45]
 [31]
 [35]
 [31]
 [30]
 [ 1]
 [27]
 [46]
 [30]] ,  (50, 1)
[29  1 31 46 45 43  1 34  1 29 38  1 41  1 46 42  1 40 35  9 38 42 46  0
 41 27  1 31  8  1 44 30 46  9 46 35 41 40 45 31 35 27 44 31 38 41 40 39
  8 45]


In [0]:
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(seq_length, batch_size)
    with tf.GradientTape() as tape:
        y_logit_pred = model(X)
        loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_logit_pred)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0: loss 4.046367
batch 1: loss 4.038974
batch 2: loss 4.036723
batch 3: loss 4.034020
batch 4: loss 4.032979
batch 5: loss 4.031949
batch 6: loss 4.025353
batch 7: loss 4.023441
batch 8: loss 4.021930
batch 9: loss 4.013144
batch 10: loss 4.015400
batch 11: loss 4.014694
batch 12: loss 4.006355
batch 13: loss 3.999507
batch 14: loss 4.002417
batch 15: loss 3.992780
batch 16: loss 3.997786
batch 17: loss 3.992878
batch 18: loss 3.972827
batch 19: loss 3.988694
batch 20: loss 3.973749
batch 21: loss 3.966169
batch 22: loss 3.959587
batch 23: loss 3.962233
batch 24: loss 3.954235
batch 25: loss 3.945163
batch 26: loss 3.939056
batch 27: loss 3.937083
batch 28: loss 3.937856
batch 29: loss 3.947582
batch 30: loss 3.933757
batch 31: loss 3.916916
batch 32: loss 3.918491
batch 33: loss 3.925578
batch 34: loss 3.902640
batch 35: loss 3.908723
batch 36: loss 3.863177
batch 37: loss 3.874146
batch 38: loss 3.849519
batch 39: loss 3.859245
batch 40: loss 3.822769
batch 41: loss 3.853086
ba

In [0]:
X_, _ = data_loader.get_batch(seq_length, 1)
for diversity in [0.2, 0.5, 1.0, 1.2]:
    X = X_
    print("diversity %f:" % diversity)
    for t in range(400):
        y_pred = model.predict(X, diversity)
        print(data_loader.indices_char[y_pred[0]], end='', flush=True)
        X = np.concatenate([X[:, 1:], np.expand_dims(y_pred, axis=1)], axis=-1)

diversity 0.200000:
e  t  net t a  ore  e t t e  t e  tt    e t te       e  t t si     i to er  t  t  t ee s e n e o e te ee tt t  t e eis l t se     ent t the  t te e e titor s ns  tl   o t  tn  s e e te  t te t  i t ta te     i t oe t t ttt stt   o  e tt s t    to   t ith t ti t tt  i e   tt toe s e e t is n eite   t in t t nt t t  t a ne t tre  t o   t int a none t  e ene tine nt  e ttse t  t e  e t  e e  aert atidiversity 0.500000:
 a iéonlit tr e nds y sehs ifo  o eefrnst ns  s ail_æ
dteetithu orde ceah  o en irthontrtacnri eotrhdsaitan noto ntcean tboie aiien ut  hr ace  i"t" stdeereee d bs eqw tefn aasge  attsas t lneo"iensi nee tre"nt rinhmtheit  i ét isaeeoooaae it mreslälai aealt iei e0toltntloaets iolto  tln eehuate nt cs  re itatr titestha a  tfit h ni t  treoon iie  taestn
pr :s wrert es stedpaederufse air   e ie todiversity 1.000000:
ä ivr;qh95_ildn69eemn"
 tældtoxc,ee.n !mo]iaraghtet"aedtnera!4nrewbx?hhl3oy9s]ileltqsuzun aghta
bonat = ro3fsallva ytutdraékahil,f; aee )
äj

/content


# 5. Tensorflow 扩展

In [174]:
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!free -g


              total        used        free      shared  buff/cache   available
Mem:             12           3           6           0           2          11
Swap:             0           0           0


In [172]:
# 授权绑定Google Drive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


E: Package 'python-software-properties' has no installation candidate
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
# 指定Google Drive云端硬盘的根目录，名为drive
!mkdir -p drive
!google-drive-ocamlfuse drive

In [186]:
import os

# 此处为google drive中的文件路径,drive为之前指定的工作根目录，要加上
os.chdir("drive/Colab_Notebooks") 

FileNotFoundError: ignored

In [182]:
f = open("file", 'w')
a = "asbd"
f.write(a)

4

In [183]:
f = open('file', 'r')
lines = f.readlines()
print(lines)

['asbd']


In [180]:
os.dir

AttributeError: ignored